In [3]:
import sqlite3

conn = sqlite3.connect("badge_attendance.db")
cursor = conn.cursor()


In [1]:
%load_ext sql

In [2]:
%sql sqlite:///badge_attendance.db

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS fact_attendance_daily (
    badge_id TEXT,
    date_id INTEGER,

    total_hours REAL,
    total_sessions INTEGER,

    first_in_time TEXT,
    last_out_time TEXT,

    avg_session_hours REAL,

    PRIMARY KEY (badge_id, date_id)
);


 * sqlite:///badge_attendance.db
Done.


[]

In [6]:
%%sql
INSERT INTO fact_attendance_daily
SELECT
    e.badge_id,
    c.date_id,

    SUM(
        (julianday(out_time) - julianday(in_time)) * 24
    ) AS total_hours,

    COUNT(*) AS total_sessions,

    MIN(in_time) AS first_in_time,
    MAX(out_time) AS last_out_time,

    AVG(
        (julianday(out_time) - julianday(in_time)) * 24
    ) AS avg_session_hours

FROM (
    SELECT
        badge_id,
        DATE(event_time) AS event_date,

        MIN(CASE WHEN event_type = 'IN' THEN event_time END) AS in_time,
        MAX(CASE WHEN event_type = 'OUT' THEN event_time END) AS out_time

    FROM src_badge_events
    GROUP BY badge_id, DATE(event_time)
) e

JOIN dim_calendar c
ON e.event_date = c.full_date

GROUP BY e.badge_id, c.date_id;


 * sqlite:///badge_attendance.db
302261 rows affected.


[]

In [7]:
%%sql
SELECT COUNT(*) FROM fact_attendance_daily;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
302261


In [8]:
%%sql
SELECT * FROM fact_attendance_daily LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,date_id,total_hours,total_sessions,first_in_time,last_out_time,avg_session_hours
BADGE000000,20240901,3.4499999955296516,1,2024-09-01T13:28:00,2024-09-01T16:55:00,3.4499999955296516
BADGE000000,20240902,6.383333332836628,1,2024-09-02T11:27:00,2024-09-02T17:50:00,6.383333332836628
BADGE000000,20240903,2.649999998509884,1,2024-09-03T10:52:00,2024-09-03T13:31:00,2.649999998509884
BADGE000000,20240904,0.7666666656732559,1,2024-09-04T11:21:00,2024-09-04T12:07:00,0.7666666656732559
BADGE000000,20240905,0.3166666701436043,1,2024-09-05T16:20:00,2024-09-05T16:39:00,0.3166666701436043
BADGE000000,20240906,1.4500000029802322,1,2024-09-06T15:02:00,2024-09-06T16:29:00,1.4500000029802322
BADGE000000,20240907,3.1166666708886623,1,2024-09-07T12:12:00,2024-09-07T15:19:00,3.1166666708886623
BADGE000000,20240909,1.3999999947845936,1,2024-09-09T15:29:00,2024-09-09T16:53:00,1.3999999947845936
BADGE000000,20240910,5.933333326131105,1,2024-09-10T10:28:00,2024-09-10T16:24:00,5.933333326131105
BADGE000000,20240912,1.300000000745058,1,2024-09-12T15:09:00,2024-09-12T16:27:00,1.300000000745058


In [9]:
%%sql
SELECT
    f.badge_id,
    c.full_date,
    f.total_hours
FROM fact_attendance_daily f
JOIN dim_calendar c
    ON f.date_id = c.date_id
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,full_date,total_hours
BADGE000000,2024-09-01,3.4499999955296516
BADGE000000,2024-09-02,6.383333332836628
BADGE000000,2024-09-03,2.649999998509884
BADGE000000,2024-09-04,0.7666666656732559
BADGE000000,2024-09-05,0.3166666701436043
BADGE000000,2024-09-06,1.4500000029802322
BADGE000000,2024-09-07,3.1166666708886623
BADGE000000,2024-09-09,1.3999999947845936
BADGE000000,2024-09-10,5.933333326131105
BADGE000000,2024-09-12,1.300000000745058


In [14]:
%%sql
SELECT
    s.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_students s
    ON f.badge_id = s.badge_id
GROUP BY s.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Anthony Johnson,1103.783333323896
Jeremy Jones,1081.0499999597669
Joshua Hamilton,1077.800000153482
Michael King,1073.9666666574776
Brandi Reed,1071.599999897182
Patricia King,1071.2333331853151
Michael Johnson,1054.9666667282581
Nicholas Smith,1048.599999871105
Nicholas Thompson,1037.3833333589137
Jennifer Smith,1031.9333332926035


In [13]:
%%sql
SELECT COUNT(*)
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
29592


In [12]:
%%sql
SELECT
    t.name AS teacher_name,
    t.department AS subject,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id
GROUP BY t.name, t.department
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


teacher_name,subject,total_hours
Zachary Chan,Arts,1121.266666688025
Larry Campbell,Engineering,1099.86666662246
Michael Spencer,Law,1092.5833333209157
Kaitlyn Miller,Law,1088.7333333604038
Mark Andrews,Engineering,1087.4666665568948
Emily Ramos,Engineering,1085.7499999031425
Denise Hernandez,Business,1085.349999960512
Linda Wells,Law,1083.7666667215526
Terry Mendez,Law,1081.6166665926576
Crystal Williams,Science,1081.4999999776483


In [11]:
%%sql
SELECT
    t.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t ON f.badge_id = t.badge_id
JOIN dim_calendar c ON f.date_id = c.date_id
WHERE c.year = 2024 AND c.month = 9
GROUP BY t.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Jacob Cervantes,182.28333332017064
Elizabeth Page,180.1999999769032
Zachary Chan,176.31666668504477
Francisco Whitaker,171.63333331793547
Tyler Martinez,169.183333363384
Alison Lang,165.98333329707384
Mary Thomas,164.71666667237878
Isabel Gregory,164.5833332799375
Sara Butler,164.48333331942558
Stephanie Newman,164.19999995827675
